<a href="https://colab.research.google.com/github/tinginde/kgc/blob/main/Lit48_Numericaltriples_Preprocessing_ver02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data preprocessing 資料前處理

Cleaning numerical literals 將資料夾中numerical literals清理

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = "/content/drive/MyDrive/litwd48k"

最新版: contains a total of 301,477 numerical triples divided in 166 different numerical attributes.(each attribute at least 15 triples)

# From LiterallyWikidata read data and check data

In [ ]:
column_name = ["e","a","v"]
df48_ori = pd.read_csv(f"{path}/lit48/numeric_literals_ori48k.txt",sep="\t",names=column_name)

In [ ]:
df48_ori[:5]

,e,a,v
0,Q1000056,P1082_Q199,+11032^^<http://www.w3.org/2001/XMLSchema#deci...
1,Q1000056,P2044_Q11573,+472^^<http://www.w3.org/2001/XMLSchema#decimal>
2,Q1000056,P2046_Q25343,+45630000.00^^<http://www.w3.org/2001/XMLSchem...
3,Q1000138,P1082_Q199,+1375^^<http://www.w3.org/2001/XMLSchema#decimal>
4,Q1000138,P2044_Q11573,+1^^<http://www.w3.org/2001/XMLSchema#decimal>


In [ ]:
df48 = df48_ori

In [ ]:
#取V值
df48[['v','xml']] = df48['v'].str.split('^', 1, expand=True)

In [ ]:
# check the number of data
print(f'triple number of df48: {len(df48)}, num of df48_entity: {len(df48["e"].unique())}, num of df48_attribute: {len(df48["a"].unique())}')

triple number of df48: 324418, num of df48_entity: 47998, num of df48_attribute: 291


In [ ]:
#有三種數值型態
df48['xml'].unique()

array(['^<http://www.w3.org/2001/XMLSchema#decimal>',
       '^<http://www.w3.org/2001/XMLSchema#dateTime>',
       '^<http://www.w3.org/2001/XMLSchema#double>'], dtype=object)

# Discovering some issues within data

In [ ]:
# same entity has multiple values 可能同entity在P1087有多個值
df48[df48["a"].str.contains("P1087_Q199")][:5]

,e,a,v,xml
1981,Q102664,P1087_Q199,+2705,^<http://www.w3.org/2001/XMLSchema#decimal>
3219,Q104148,P1087_Q199,+2480,^<http://www.w3.org/2001/XMLSchema#decimal>
3220,Q104148,P1087_Q199,+2485,^<http://www.w3.org/2001/XMLSchema#decimal>
3221,Q104148,P1087_Q199,+2486,^<http://www.w3.org/2001/XMLSchema#decimal>
3222,Q104148,P1087_Q199,+2491,^<http://www.w3.org/2001/XMLSchema#decimal>


In [ ]:
# coordinate location in original data is mixed. P625，P1335...(coordinate location)有經緯度
df48[df48["a"].str.contains("P1335")]

,e,a,v,xml
206346,Q100015,P1335_Latitude,45.5439988,^<http://www.w3.org/2001/XMLSchema#double>
206351,Q100016,P1335_Latitude,45.8495584,^<http://www.w3.org/2001/XMLSchema#double>
206356,Q100018,P1335_Latitude,45.6671782,^<http://www.w3.org/2001/XMLSchema#double>
206362,Q100036,P1335_Latitude,44.7203568,^<http://www.w3.org/2001/XMLSchema#double>
206367,Q100038,P1335_Latitude,44.7499868,^<http://www.w3.org/2001/XMLSchema#double>
...,...,...,...,...
324394,Q99981,P1335_Longtiude,9.4935722,^<http://www.w3.org/2001/XMLSchema#double>
324399,Q99983,P1335_Longtiude,9.6883379,^<http://www.w3.org/2001/XMLSchema#double>
324404,Q99985,P1335_Longtiude,9.7203429,^<http://www.w3.org/2001/XMLSchema#double>
324409,Q99986,P1335_Longtiude,9.5321288,^<http://www.w3.org/2001/XMLSchema#double>


In [ ]:
# P2299 has 2 units 有兩種QXXX
df48[df48["a"].str.contains("P2299")].a.unique()

array(['P2299_Q550207', 'P2299_Q4917'], dtype=object)

In [ ]:
# preparing idmap for checking real name
entity_idmap = {}
with open(f'{path}/entity_labels_en.txt',encoding="utf-8") as f:
    for line in f:
        k, v = line.strip().split('\t')
        entity_idmap[k.strip()] = v.strip()
attri_idmap={}
with open(f'{path}/attribute_labels_en.txt',encoding="utf-8") as f:
    for line in f:
        k, v = line.strip().split('\t')
        attri_idmap[k.strip()] = v.strip()
attri_idmap['P625_Latitude']='coordinate location(latitude)'
attri_idmap['P625_Longtiude']='coordinate location(logtitude)'
attri_idmap['P1332_Longtiude']='coordinates of northernmost point'
attri_idmap['P1333_Longtiude']='coordinates of southernmost point'
attri_idmap['P1334_Longtiude']='coordinates of easternmost point'
attri_idmap['P1335_Longtiude']='coordinates of westernmost point'

In [ ]:
# some are meanful zero value but some are not
df_vzero=df48[df48["v"]==0].sort_values(by='a')
df_vzero.loc[:,'name_e']=df48['e'].map(entity_idmap)
df_vzero.loc[:,'name_a']=df48['a'].map(attri_idmap)

In [ ]:
df_vzero

,e,a,v,xml,new_v,pre_a,Q,name_e,name_a
29269,Q1818674,P1082,0.0,^<http://www.w3.org/2001/XMLSchema#decimal>,+0,P1082,Q199,Sule Stack,population
24010,Q1655937,P1082,0.0,^<http://www.w3.org/2001/XMLSchema#decimal>,+0,P1082,Q199,St. Matthew Island,population
133081,Q834577,P1082,0.0,^<http://www.w3.org/2001/XMLSchema#decimal>,+0,P1082,Q199,Vot Tande,population
101793,Q5552475,P1082,0.0,^<http://www.w3.org/2001/XMLSchema#decimal>,+0,P1082,Q199,Gerovasa,population
98936,Q540373,P1082,0.0,^<http://www.w3.org/2001/XMLSchema#decimal>,+0,P1082,Q199,Zemlya Georga,population
...,...,...,...,...,...,...,...,...,...
91615,Q5115598,P6547,0.0,^<http://www.w3.org/2001/XMLSchema#decimal>,+0,P6547,Q199,Chuck Luksa,career plus-minus rating
100611,Q549201,P6547,0.0,^<http://www.w3.org/2001/XMLSchema#decimal>,+0,P6547,Q199,Matt Read,career plus-minus rating
49627,Q2742741,P6547,0.0,^<http://www.w3.org/2001/XMLSchema#decimal>,+0,P6547,Q199,Brian Felsner,career plus-minus rating
124904,Q739767,P6547,0.0,^<http://www.w3.org/2001/XMLSchema#decimal>,+0,P6547,Q199,Harijs Vītoliņš,career plus-minus rating


In [ ]:
print(f"there are total {len(df_vzero)} data which has 0.0 in the value column")
print(f"there are total {len(df_vzero['a'].unique())} data which has 0.0 in the value column")

there are total 2528 data which has 0.0 in the value column
there are total 35 data which has 0.0 in the value column


In [ ]:
#check where df['v']=0 is
df48[(df48['v']==0.0) & (df48['a']=='P1120')]
df48[df48['a']=='P5045']

,e,a,v,xml,new_v,pre_a,Q
5778,Q10990655,P5045,6219.0,^<http://www.w3.org/2001/XMLSchema#decimal>,+621,P5045,Q199
9140,Q117868,P5045,52898.0,^<http://www.w3.org/2001/XMLSchema#decimal>,+528,P5045,Q199
9806,Q1204824,P5045,47608.0,^<http://www.w3.org/2001/XMLSchema#decimal>,+476,P5045,Q199
10147,Q1228693,P5045,99161.0,^<http://www.w3.org/2001/XMLSchema#decimal>,+991,P5045,Q199
10966,Q125961,P5045,91631.0,^<http://www.w3.org/2001/XMLSchema#decimal>,+916,P5045,Q199
...,...,...,...,...,...,...,...
120160,Q7098220,P5045,34083.0,^<http://www.w3.org/2001/XMLSchema#decimal>,+340,P5045,Q199
126130,Q751553,P5045,25373.0,^<http://www.w3.org/2001/XMLSchema#decimal>,+253,P5045,Q199
136695,Q863595,P5045,43269.0,^<http://www.w3.org/2001/XMLSchema#decimal>,+432,P5045,Q199
137807,Q8774386,P5045,17960.0,^<http://www.w3.org/2001/XMLSchema#decimal>,+179,P5045,Q199


In [ ]:
df_vzero[df_vzero['name_a']=='attendance']

,e,a,v,xml,new_v,pre_a,Q,name_e,name_a
6677,Q111,P1110,0.0,^<http://www.w3.org/2001/XMLSchema#decimal>,+0,P1110,Q199,Mars,attendance


# Dealing with issues 處理以上問題: 
Take xml part away "v"有"^^<http...>"RDF格式部分，先拿掉 \\

Turn into value dtype to float 
所有變數的值成為可計算的值(有意義可讀取的值) \\

DataTime dtype: keep year data 時間單位 西元年 \\
Rescale the values within the same attributes 同attribute value rescale \\
Drop out unreasonal 0 value 去0 \\
Drop out the unit (QXXX)去單位 \\
Keep Attribyes at least number of 15 data 每個屬性最少有15筆資料 \\


In [ ]:
# 如果"v"是時間，取前四位數年代部分，np.where(condition, x,y)在condition下，把x取代y
df48["new_v"] = df48["v"].map(lambda x: x[:4])
df48.loc[:,"v"] = np.where((df48["xml"].str.contains("dateTime")), df48["new_v"], df48["v"])

In [ ]:
df48["v"]=df48["v"].astype('float')

In [ ]:
df48.dtypes

e         object
a         object
v        float64
xml       object
new_v     object
dtype: object

In [ ]:
#處理a，去單位
df48[["pre_a",'Q']] = df48["a"].str.split('_', 1,expand=True)
df48.loc[:,"a"] = np.where((df48["a"].str.contains("Q")), df48["pre_a"], df48["a"])

In [ ]:
df=df48

In [ ]:
# 刪掉特定列，有11個不合理的0值變數
df = df.drop(df[(df['a']=='P6509') & (df['v']==0.0)].index)#進球相關
df = df.drop(df[(df['a']=='P6545') & (df['v']==0.0)].index)
df = df.drop(df[(df['a']=='P6544') & (df['v']==0.0)].index)
df = df.drop(df[(df['a']=='P6546') & (df['v']==0.0)].index)
df = df.drop(df[(df['a']=='P6543') & (df['v']==0.0)].index)
df = df.drop(df[(df['a']=='P1279') & (df['v']==0.0)].index)#inflation rate
df = df.drop(df[(df['a']=='P2121') & (df['v']==0.0)].index)#prize money
df = df.drop(df[(df['a']=='P6344') & (df['v']==0.0)].index)#rural population
df = df.drop(df[(df['a']=='P569') & (df['v']==0.0)].index)#birthYear
df = df.drop(df[(df['a']=='P3872') & (df['v']==0.0)].index)#惠顧人數patronage
df = df.drop(df[(df['a']=='P6547') & (df['v']==0.0)].index)#career plus-minus rating

## Rescale decimal values

In [ ]:
def attvalue2scale(df):
  alist=list(df['a'].unique())
  for a_name in alist:
    attdf = df[df["a"]==a_name]
    for i in range(len(attdf)):
      result = (df.v - attdf.v.min())/ (attdf.v.max()- attdf.v.min())
      return result


In [ ]:
df.loc[:,"rescale_v"] = np.where((df['xml'].str.contains('decimal')), attvalue2scale(df), df['v'])

In [ ]:
df48_meta = df.loc[:,['e','a','v','xml','rescale_v']]

In [ ]:
df48_meta.to_csv(f'{path}/cleaningdata/df48_meta',sep="\t",index=False)

In [ ]:
df48_s = df48_meta.loc[:,['e','a','rescale_v']]

In [ ]:
#處理重複資料
df48_s = df48_s.groupby(['e', 'a'], as_index=False).mean()

In [ ]:
#keep: attributes > 15
filter = df48_s["a"].value_counts()
df48_s = df48_s[df48_s['a'].isin(filter.index[filter>=15])]

In [ ]:
print(f'triple number of df48: {len(df48_s)}, num of df48_entity: {len(df48_s["e"].unique())}, num of df48_attribute: {len(df48_s["a"].unique())}')

triple number of df48: 301601, num of df48_entity: 47988, num of df48_attribute: 166


## Save the cleaning data 存檔

In [ ]:
df48_s

,e,a,rescale_v
0,Q100,P1082,1.483085e-04
1,Q100,P2044,1.032598e-08
2,Q100,P2046,5.574606e-02
3,Q100,P571,1.630000e+03
4,Q100,P625_Latitude,4.235833e+01
...,...,...,...
302005,Q99987,P1335_Longtiude,9.564391e+00
302006,Q99987,P2044,6.411716e-08
302007,Q99987,P2046,9.941761e-04
302008,Q99987,P625_Latitude,4.571767e+01


In [ ]:
df48_s.to_csv("/content/drive/MyDrive/litwd48k/cleaningdata/numeric_literals_final_ver03",sep="\t",index=False)

## Split into train, valid and test sets


In [ ]:
df = pd.read_csv(f'{path}/cleaningdata/numeric_literals_final_ver03',sep='\t')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301601 entries, 0 to 301600
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   e          301601 non-null  object 
 1   a          301601 non-null  object 
 2   rescale_v  301601 non-null  float64
dtypes: float64(1), object(2)
memory usage: 6.9+ MB


In [ ]:
from sklearn.model_selection import train_test_split
train_attri_data, valid_attri_data = train_test_split(df, test_size=0.2,stratify=df['a'],random_state=802)
valid_attri_data, test_attri_data = train_test_split(valid_attri_data, test_size=0.5,stratify=valid_attri_data['a'],random_state=802)

In [ ]:
print(f'training triples of df48: {len(train_attri_data)}, valid triples of df48: {len(valid_attri_data)}, test triples of df48: {len(test_attri_data)}')

training triples of df48: 241280, valid triples of df48: 30160, test triples of df48: 30161


In [ ]:
test_attri_data.a.value_counts()

P625_Latitude     2496
P625_Longtiude    2495
P2046             2175
P1082             2145
P2044             1642
                  ... 
P1114                1
P2999                1
P2386                1
P1339                1
P1314                1
Name: a, Length: 166, dtype: int64

In [ ]:
train_attri_data.a.value_counts()

P625_Longtiude    19962
P625_Latitude     19961
P2046             17401
P1082             17157
P2044             13134
                  ...  
P1114                14
P1339                13
P2386                12
P2999                12
P1314                12
Name: a, Length: 166, dtype: int64

In [ ]:
train_attri_data.to_csv(f'{path}/cleaningdata/train_attri_data',index=False)
valid_attri_data.to_csv(f'{path}/cleaningdata/valid_attri_data',index=False)
test_attri_data.to_csv(f'{path}/cleaningdata/test_attri_data',index=False)

# make array for training data

In [ ]:
df_train_att= pd.read_csv(f'{path}/cleaningdata/train_attri_data')

NameError: ignored

In [ ]:
df_train_att.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241280 entries, 0 to 241279
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   e          241280 non-null  object 
 1   a          241280 non-null  object 
 2   rescale_v  241280 non-null  float64
dtypes: float64(1), object(2)
memory usage: 5.5+ MB


In [ ]:
train_num_lit = np.zeros([len(df_train_att['e']), len(df_train_att['a'])], dtype=np.float32)
